In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Omitted Part

In [ ]:
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt

def create_spectrogram(audio_path, output_path):
    y, sr = librosa.load(audio_path, sr=None)
    D = np.abs(librosa.stft(y))
    D_db = librosa.amplitude_to_db(D, ref=np.max)
    fig, ax = plt.subplots(figsize=(28, 5))
    ax.axis('off')
    librosa.display.specshow(D_db, sr=sr, x_axis='time', y_axis='log', ax=ax)
    plt.savefig(output_path, bbox_inches='tight', pad_inches=0)
    plt.close()

# Path to the WAV file
wav_file_path = "/content/drive/MyDrive/AUDIO/New Model_Main/Real_40/Copy of YONGEY_MINGYU_part8.wav"

# Output path for the spectrogram image
output_img_path = "/content/drive/MyDrive/AUDIO/New Model_Main/Spectro_img/Real/Copy of YONGEY_MINGYU_part8.png"

# Create the spectrogram image
create_spectrogram(wav_file_path, output_img_path)

In [ ]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import os

def preprocess_and_save_spectrogram(input_dir, output_dir):
    # Create the output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Iterate over WAV files in the input directory
    for file_name in os.listdir(input_dir):
        if file_name.endswith('.wav'):
            # Load the audio file
            file_path = os.path.join(input_dir, file_name)
            y, sr = librosa.load(file_path)


            # Generate the spectrogram
            spectrogram = librosa.feature.melspectrogram(y=y, sr=sr)
            spectrogram_db = librosa.power_to_db(spectrogram, ref=np.max)

            # Plot the spectrogram
            plt.figure(figsize=(10, 5))
            librosa.display.specshow(spectrogram_db, sr=sr, x_axis='time', y_axis='mel')
            plt.axis('off')  # Turn off axis
            plt.savefig(os.path.join(output_dir, file_name[:-4] + '.png'), bbox_inches='tight', pad_inches=0)
            plt.close()


# Preprocess real audio files
preprocess_and_save_spectrogram('/content/drive/MyDrive/AUDIO/New Model_Main/Real_40','/content/drive/MyDrive/AUDIO/New Model_Main/test')

print("Spectrogram images generated successfully.")

KeyboardInterrupt: 

# Final Model Building


In [ ]:
train_dir = '/content/drive/MyDrive/AUDIO/spectrogram_images/Train_Audio'
test_dir = '/content/drive/MyDrive/AUDIO/spectrogram_images/Test_Audio'

In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [ ]:
def load_images_from_directory(directory):
    images = []
    labels = []
    for label, class_name in enumerate(['Fake', 'Real']):
        class_dir = os.path.join(directory, class_name)
        for filename in os.listdir(class_dir):
            if filename.endswith('.png'):
                img = load_img(os.path.join(class_dir, filename), target_size=(775, 385))
                img_array = img_to_array(img)
                images.append(img_array)
                labels.append(label)
    return np.array(images), np.array(labels)

In [ ]:
train_audio, train_labels = load_images_from_directory(train_dir)

In [ ]:
test_audio, test_labels = load_images_from_directory(test_dir)

In [ ]:
train_audio = train_audio.astype('float32') / 255.0
test_audio = test_audio.astype('float32') / 255.0

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_audio, val_audio, train_labels, val_labels = train_test_split(train_audio, train_labels, test_size=0.2, random_state=42)

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam

In [ ]:
model = Sequential([
    Flatten(input_shape=(775, 385, 3)),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])

In [ ]:
model = Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0001), metrics=['accuracy'])

In [ ]:
model.fit(train_audio, train_labels, validation_data=(val_audio, val_labels), epochs=15, batch_size=32)

Epoch 1/15
23/23 [==============================] - 28s 1s/step - loss: 0.3620 - accuracy: 0.7972 - val_loss: 0.2130 - val_accuracy: 0.9333
Epoch 2/15
23/23 [==============================] - 23s 1s/step - loss: 0.3727 - accuracy: 0.7833 - val_loss: 0.3386 - val_accuracy: 0.8056
Epoch 3/15
23/23 [==============================] - 23s 1s/step - loss: 0.4146 - accuracy: 0.7153 - val_loss: 0.2075 - val_accuracy: 0.9222
Epoch 4/15
23/23 [==============================] - 23s 1s/step - loss: 0.3819 - accuracy: 0.7375 - val_loss: 0.2056 - val_accuracy: 0.9278
Epoch 5/15
23/23 [==============================] - 23s 1s/step - loss: 0.4548 - accuracy: 0.7083 - val_loss: 0.2658 - val_accuracy: 0.9333
Epoch 6/15
23/23 [==============================] - 23s 1s/step - loss: 0.5575 - accuracy: 0.7014 - val_loss: 0.4065 - val_accuracy: 0.9278
Epoch 7/15
23/23 [==============================] - 23s 1s/step - loss: 0.4772 - accuracy: 0.7472 - val_loss: 0.2874 - val_accuracy: 0.8444
Epoch 8/15
23/23 [==

In [ ]:
loss, accuracy = model.evaluate(test_audio, test_labels)

print(f"Test loss: {loss}")
print(f"Test accuracy: {accuracy}")

7/7 [==============================] - 6s 828ms/step - loss: 0.3528 - accuracy: 0.8909
Test loss: 0.352847695350647
Test accuracy: 0.8909090757369995


In [ ]:
from keras.models import load_model
from PIL import Image
import numpy as np

In [ ]:
# Model_file = '/content/drive/MyDrive/AUDIO/New Model_Main/vgg_based.h5'

In [ ]:
# model =  load_model(Model_file)

In [ ]:
image_path = '/content/drive/MyDrive/AUDIO/spectrogram_images/REAL/Abdul_kalam_part3.png'
img = load_img(image_path, target_size=(775, 385))
img_array = img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = img_array / 255.0

In [ ]:
probabilities = model.predict(img_array)[0]

1/1 [==============================] - 0s 63ms/step


In [ ]:
# Predict the class probabilities
probabilities = model.predict(img_array)[0]

if len(probabilities) == 1:
    print(f"Probability of being Fake: {1 - probabilities[0]}")
    print(f"Probability of being Real: {probabilities[0]}")
else:
    print(f"Probability of being Fake: {probabilities[0]}")
    print(f"Probability of being Real: {probabilities[1]}")

1/1 [==============================] - 0s 72ms/step
Probability of being Fake: 0.0011066794395446777
Probability of being Real: 0.9988933205604553


In [ ]:
input_layer = model.layers[0].input

In [ ]:
input_layer.shape

TensorShape([None, 775, 385, 3])

In [ ]:
model.save('/content/drive/MyDrive/AUDIO/deepfake_final.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
